<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

##  CPU Trigger Test


Test CPU Trigger to activate Channel operation.

Test Setup:
* One module of P948x
* Scope connected to channel 1 and 3  of the module.

This example creates 3 waves of different types and downloads them into a task table per channel, the task's wait for an  internal CPU trigger to activate the operation. The coupled cpu trigger mode activates all task tables together.
Run script and verify on Scope that after the cpu trigger command ('*TRG') signals are generated continuosly.

In [ ]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [ ]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [ ]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.42::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

In [ ]:
# Set the default paranoia level. 
# It controls the default behavior of: inst.send_scpi_cmd(scpi_str).
#  - paranoia-level = 0: send scpi_str as is.
#  - paranoia-level = 1: append '*OPC?' to scpi_str, and send it as query 
#  - paranoia-level = 2: append ':SYST:ERR?' to scpi_str, send it as query, and print warnning in case of error.

inst.default_paranoia_level = 2 # paranoia-level = 2 (good for debugging)

# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)

In [ ]:
#Set the COUPLE flag for the script.
COUPLE=True
inst.send_scpi_cmd(':INST:CHAN 1')

if COUPLE:
  inst.send_scpi_cmd(':TRIG:COUPLE ON')
  inst.send_scpi_cmd(':TRIG:CPU:MODE GLOBAL')
  #inst.send_scpi_cmd(':TRIG:CPU:MODE LOCAL')

## Test Signal generatation from the task table.
#inst.send_scpi_cmd('*TRG')


In [ ]:
# Set sampling clock rate
inst.send_scpi_cmd(':FREQ:RAST 2.5e9')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

In [ ]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    channels_per_dac = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
    
half_dac = max_dac / 2.0

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = (int(resp) // 64) * 64
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_seglen = arbmem_capacity // bpp
print ('Max segment length: {0:,}'.format(max_seglen))

In [ ]:
# Build 3 waveforms

seglen = 4096
cyclelen = seglen
ncycles = seglen / cyclelen
waves = [ None for _ in range(3)]

# sin wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[0] = y.astype(wpt_type)

# triangle wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = np.sin(x)
y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[1] = y.astype(wpt_type)

# square wave
x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
y = np.fmod(x, cyclelen)
y = (y <= cyclelen / 2) * max_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[2] = y.astype(wpt_type)

In [ ]:
# download 3 waveforms to each DDR

for ichan in range(num_channels):
    if ichan % channels_per_dac == 0:
        channb = ichan + 1
        # Select channel
        cmd = ':INST:CHAN {0}'.format(channb)
        inst.send_scpi_cmd(cmd)
        for iseg in range(3):
            segnum = iseg + 1
            print('Downloading segment {0} of channel {1}'.format(segnum, channb))
            # Define segment
            cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
            inst.send_scpi_cmd(cmd)
            
            # Select the segment
            cmd = ':TRAC:SEL {0}'.format(segnum)
            inst.send_scpi_cmd(cmd)
            
            # Send the binary-data:
            inst.write_binary_data(':TRAC:DATA', waves[iseg])
            
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

In [ ]:
# Play the first segment in each channel

for ichan in range(num_channels):
    channb = ichan + 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(1)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

In [ ]:
# Define task-table of 3 tasks in each channel.
# The first task shall wait for trigger1.
# In order to 

tasklen = 3
if COUPLE:
   TRG_SRC = 'INT' 

# Compose the task-table rows:
cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
inst.send_scpi_cmd(cmd)

for itask in range(tasklen):
    tasknb = itask + 1
    segnb = itask + 1
    nloops = 2 ** tasknb
    
    cmd = ':TASK:COMP:SEL {0}'.format(tasknb)
    inst.send_scpi_cmd(cmd)
    
    inst.send_scpi_cmd(':TASK:COMP:TYPE SING')
    
    cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
    inst.send_scpi_cmd(cmd)
    
    cmd = ':TASK:COMP:LOOP {0}'.format(nloops)
    inst.send_scpi_cmd(cmd)
    
    if 1 == tasknb:
        # in case of :TRIG:COUPLE ON need to put INT instead of CPU??
        cmd = ':TASK:COMP:ENAB {}'.format(TRG_SRC)
        inst.send_scpi_cmd(cmd)
    else:
        cmd = ':TASK:COMP:ENAB NONE'
        inst.send_scpi_cmd(cmd)
        
    if tasklen == tasknb:
        cmd = ':TASK:COMP:NEXT1 2'
        inst.send_scpi_cmd(cmd)
    else:
        cmd = ':TASK:COMP:NEXT1 {0}'.format(tasknb + 1)
        inst.send_scpi_cmd(cmd)

# write the task table rows to the task-table of each channel
for ichan in range(num_channels):
    channb = ichan + 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    # Write the task-table
    inst.send_scpi_cmd(':TASK:COMP:WRIT')
    
    # Set Task-Mode
    inst.send_scpi_cmd(':FUNC:MODE TASK')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## Send CPU Trigger - 
Verify on Scope connected to Channel 1 that signal is generated.

In [ ]:
inst.send_scpi_cmd('*TRG')

In [ ]:
inst.close_instrument()